In [ ]:
# Melakukan Import module numpy, pandas, dan seaborn
import numpy as np # Untuk perhitungan yang berkaitan dengan matematika
import pandas as pd # Untuk read csv dan processing data pada dataframe
import seaborn as sns # Untuk membuat grafik dari data yang telah diproses
import matplotlib.pyplot as plt
sns.set(style="darkgrid")

import warnings
warnings.filterwarnings('ignore')

from scipy import stats # Untuk mempermudah perhitungan statistika
from datetime import * # Untuk memudahkan partisi pada time

# Memudahkan import csv dari input kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Melakukan import csv dari data-data yang disediakan dengan panda.read_csv

gd = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv')

en_test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')
en_train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')

cu_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')
cu_train = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')

In [ ]:
gd.head() # Melihat secara sekilas data bulding_metadata.csv

In [ ]:
# Melihat informasi data frame bulding_metadata.csv,seperti banyaknya row dan column, type data setiap variabel
gd.info()

# Melihat jumlah missing values pada data frame bulding_metadata.csv
gd.isnull().sum()

In [ ]:
# Membuat fungsi untuk melakukan reduce pada data untuk meringankan program agar tetap dapat berjalan

def reduce_mem_usage(df_train):
    start_mem = df_train.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    for col in df_train.columns:
        col_type = df_train[col].dtype

        if col_type != object:
            c_min = df_train[col].min()
            c_max = df_train[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df_train[col] = df_train[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df_train[col] = df_train[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df_train[col] = df_train[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df_train[col] = df_train[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df_train[col] = df_train[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df_train[col] = df_train[col].astype(np.float32)
                else:
                    df_train[col] = df_train[col].astype(np.float64)
        else:
            df_train[col] = df_train[col].astype('category')

    end_mem = df_train.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df_train

PREPROCESSING DATA TRAIN

In [ ]:
# Melihat secara seklilas data pada train.csv
en_train.head()

In [ ]:
# Melihat informasi data frame train.csv,seperti banyaknya row dan column, type data setiap variabel
en_train.info()

# Melihat jumlah missing values pada data frame train.csv
en_train.isnull().sum()

In [ ]:
# Melakukan left join data test.csv dengan building_metadata.csv pada column building_id
train_en_gd = en_train.merge(gd, how = 'left', on = 'building_id')

# Melihat secara seklilas data pada gabungan/hasil left join data test.csv dengan building_metadata.csv
train_en_gd.head()

In [ ]:
# Melihat secara seklilas data pada weather_train.csv
cu_train.head()

In [ ]:
# Melihat informasi pada data frame weather_train.csv
cu_train.info()

# Melihat jumlah missing values pada data frame weather_train.csv
cu_train.isnull().sum()

In [ ]:
# Melakukan cek pola missing values yang terdapat pada cu_train

sns.heatmap(cu_train.isnull(), cbar=False)

In [ ]:
# Missing values dapat diisi dengan median dari suatu data pada site tertentu dan pada waktu tertentu
# Untuk memudahkan, waktu yang diambil adalah waktu hours setiap harinya
# Mengambil timestamp asli, sebelum dilakukan manipulasi
cu_train_timestamp = cu_train['timestamp']
cu_train_timestamp = cu_train_timestamp.to_frame()

In [ ]:
# Mengisi missing values pada data yang berhubungan dengan weather
# Mengambil hours dari timestamp untuk nanti mencari median dari weather pada hours dan pada site tertentu
# Memanipulasi timestamp dengan menggantinya menjadi hours
cu_train['timestamp'] = pd.to_datetime(cu_train['timestamp'])
cu_train['timestamp'] = pd.to_datetime(cu_train['timestamp'], format = "%Y-%m-%d %H:%M:%S").dt.strftime('%m,%H')

In [ ]:
# Program dibawah ini akan membuat column yang berisi median dari data-data pada jam dan site tertentu
# Nantinya column yang telah dibuat akan digunakan untuk mengisi missing values dengan median sesuai dengan variabelnya

cu_traina = cu_train.groupby(['site_id','timestamp'])['air_temperature'].median()
cu_traina = cu_traina.to_frame()
cu_train = cu_train.merge(cu_traina, how = 'left', on = ['timestamp','site_id'])
cu_train['air_temperature_x'] = cu_train['air_temperature_x'].fillna(cu_train['air_temperature_y'])

cu_trainc = cu_train.groupby(['site_id','timestamp'])['cloud_coverage'].median()
cu_trainc = cu_trainc.to_frame()
cu_train = cu_train.merge(cu_trainc, how = 'left', on = ['timestamp','site_id'])
cu_train['cloud_coverage_x'] = cu_train['cloud_coverage_x'].fillna(cu_train['cloud_coverage_y'])

cu_traind = cu_train.groupby(['site_id','timestamp'])['dew_temperature'].median()
cu_traind = cu_traind.to_frame()
cu_train = cu_train.merge(cu_traind, how = 'left', on = ['timestamp','site_id'])
cu_train['dew_temperature_x'] = cu_train['dew_temperature_x'].fillna(cu_train['dew_temperature_y'])

cu_trainp = cu_train.groupby(['site_id','timestamp'])['precip_depth_1_hr'].median()
cu_trainp = cu_trainp.to_frame()
cu_train = cu_train.merge(cu_trainp, how = 'left', on = ['timestamp','site_id'])
cu_train['precip_depth_1_hr_x'] = cu_train['precip_depth_1_hr_x'].fillna(cu_train['precip_depth_1_hr_y'])

cu_trains = cu_train.groupby(['site_id','timestamp'])['sea_level_pressure'].median()
cu_trains = cu_trains.to_frame()
cu_train = cu_train.merge(cu_trains, how = 'left', on = ['timestamp','site_id'])
cu_train['sea_level_pressure_x'] = cu_train['sea_level_pressure_x'].fillna(cu_train['sea_level_pressure_y'])

cu_trainwd = cu_train.groupby(['site_id','timestamp'])['wind_direction'].median()
cu_trainwd = cu_trainwd.to_frame()
cu_train = cu_train.merge(cu_trainwd, how = 'left', on = ['timestamp','site_id'])
cu_train['wind_direction_x'] = cu_train['wind_direction_x'].fillna(cu_train['wind_direction_y'])

cu_trainws = cu_train.groupby(['site_id','timestamp'])['wind_speed'].median()
cu_trainws = cu_trainws.to_frame()
cu_train = cu_train.merge(cu_trainws, how = 'left', on = ['timestamp','site_id'])
cu_train['wind_speed_x'] = cu_train['wind_speed_x'].fillna(cu_train['wind_speed_y'])

In [ ]:
# Karena timestamp masih berbentuk jam hasil manipulasi, lakukan join timestamp awal yang disimpan
cu_train = pd.merge(cu_train, cu_train_timestamp, left_index=True, right_index=True)

# Karena masih terdapat column yang berisi median pada setiap variabel, lakukan drop column tersebut
cu_train = cu_train.drop(columns = ['timestamp_x','air_temperature_y',
                                  'cloud_coverage_y','dew_temperature_y',
                                  'precip_depth_1_hr_y','sea_level_pressure_y',
                                  'wind_direction_y', 'wind_speed_y'])

# Mengurutkan column seperti awal dataframe dan melakukan rename column seperti awal
cu_train = cu_train[['site_id', 'timestamp_y', 'air_temperature_x','cloud_coverage_x', 'dew_temperature_x', 
                    'precip_depth_1_hr_x', 'sea_level_pressure_x','wind_direction_x', 'wind_speed_x']]
cu_train = cu_train.rename(columns={"timestamp_y": "timestamp","air_temperature_x":"air_temperature",
                        "cloud_coverage_x": "cloud_coverage","dew_temperature_x": "dew_temperature",
                        "precip_depth_1_hr_x": "precip_depth_1_hr","sea_level_pressure_x": "sea_level_pressure",
                        "wind_direction_x": "wind_direction","wind_speed_x": "wind_speed"})

In [ ]:
# Melakukan cek apakah missing values masih terdapat dalam data
cu_train.isnull().sum()

In [ ]:
# Melakukan cek pola missing values yang terdapat pada cu_train

sns.heatmap(cu_train.isnull(), cbar=False)

In [ ]:
# Missing values yang masih terdapat pada cloud_coverage dapat dilakukan fillna(0)
# Dengan asumsi bahwa missing values terjadi karena tidak adanya cloud pada waktu tersebut, sehingga dapat diisi 0
cu_train['cloud_coverage'] = cu_train['cloud_coverage'].fillna(0)

# Missing values yang masih terdapat pada precip_depth_1_hr dapat dilakukan fill dengan median dari data tersebut
median_precip = cu_test['precip_depth_1_hr'].median()
cu_train['precip_depth_1_hr'] = cu_train['precip_depth_1_hr'].fillna(median_precip)

# Missing values yang masih terdapat pada sea_level_pressure dapat dilakukan fill dengan median dari data tersebut
median_sea = cu_test['sea_level_pressure'].median()
cu_train['sea_level_pressure'] = cu_train['sea_level_pressure'].fillna(median_sea)

In [ ]:
# Melakukan cek persentase missing values pada data
cu_train.count()/len(cu_train)

In [ ]:
# Melakukan reduce untuk data rain_en_gd dan cu_train
train_en_gd = reduce_mem_usage(train_en_gd)
cu_train = reduce_mem_usage(cu_train)

In [ ]:
# Melakukan left join data gabungan pada train_en_gd dengan data cuaca weather_train pada column timestamp dan site_id
df_train = train_en_gd.merge(cu_train, how = 'left', on = ['site_id','timestamp'])

# Melihat sekilas pada data frame df_test yang telah digabung dari 3 dataframe
df_train.head()

In [ ]:
# Melihat informasi data frame gabungan
df_train.info()

# Melihat persentase missing values pada data frame df_train
df_train.count()/len(df_train)

In [ ]:
# Mencari nilai yang menjadi missing values
df_train_null = df_train[df_train.air_temperature.isnull()]
df_train_null.head()

In [ ]:
# Missing values terjadi karena ketiadaan data pada suatu rentang waktu timestamp weather_train.csv
# Missing values pada tahap ini dapat dilakukan dropna() 
df_train = df_train.dropna(subset=['air_temperature']) 

In [ ]:
# Memastikan missing values telah di drop
print(df_train.isnull().sum())

# Melihat kembali info dari data
print(df_train.info())

In [ ]:
# Mengganti type data dari column timestamp dari object menjadi date time karena merupakan data time
df_train['timestamp'] = df_train['timestamp'].astype('object')
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])

#Melihat informasi data frame untuk melihat apakah type data timestamp sudah terganti menjadi date time
df_train.info()

In [ ]:
# Melakukan cek sekilas kekonsistenan dari beberapa variabel yang memungkinkan terdapat ketidakkonsistenan

tr_buildid = df_train['building_id'].unique()
tr_buildid.sort()
print(tr_buildid)

tr_met = df_train['meter'].unique()
tr_met.sort()
print(tr_met)

tr_siteid = df_train['site_id'].unique()
tr_siteid.sort()
print(tr_siteid)

tr_primaryu = df_train['primary_use'].unique()
print(tr_primaryu)

tr_yearb = df_train['year_built'].unique()
tr_yearb.sort()
print(tr_yearb)

tr_floorc = df_train['floor_count'].unique()
tr_floorc.sort()
print(tr_floorc)

Missing values pada year_built dan floor_count pada data df_train dapat dihiraukan, karena bukan merupakan poin penting dalam melakukan analisa data penggunaan energi

Mengatasi outlier pada data train karena akan dilakukan analisa statistika dasar

In [ ]:
# Melakukan cek distribusi meter reading dengan group by site id
df_train_gb_site = df_train.groupby('site_id').meter_reading.mean().reset_index()
sns.distplot(df_train_gb_site)

In [ ]:
# Melakukan cek terhadap grafik rata-rata meter reading dari seteiap site
sns.barplot(x='site_id',y='meter_reading',data = df_train_gb_site)

In [ ]:
# Melakukan cek terhadap analisa statistika dasar dari site_id 13
df_train_site13 = df_train.loc[df_train['site_id'] == 13]
df_train_site13.describe()

In [ ]:
# Melakukan sort data berdasarkan meter reading secara descending

df_train.sort_values('meter_reading', ascending=False)

In [ ]:
# Karena terdapat banyak sekali data outlier berasal dari building_id 1099, melakukan cek terhadap data tersebut

df_train_id1099 = df_train.loc[df_train['building_id'] == 1099]
df_train_id1099.describe()

In [ ]:
# Membuat fungsi yang dapat mendeteksi outlier dengan konsep IQR quantile tetapi hanya pada data uppernya dan nilai dari upper_b nya

def outl_up(df,col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_b = Q1 - 1.5*IQR
    upper_b = Q3 + 1.5*IQR
    outl_up_data = df.loc[(df[col] > upper_b)]
    return outl_up_data

def outl_up_nilai(df,col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_b = Q1 - 1.5*IQR
    upper_b = Q3 + 1.5*IQR
    outl_up_data = df.loc[(df[col] > upper_b)]
    return upper_b

In [ ]:
# Mencari dasar statistika pada data yang menjadi outlier upper dan memasukkan nilai upper_b nya

outl_up_meter_read = outl_up(df_train_id1099,'meter_reading')
outl_up_meter_read_b = outl_up_nilai(df_train_id1099,'meter_reading')

outl_up_meter_read.describe()

In [ ]:
# Melakukan drop data outlier pada meter_reading di building_id 1099

len_raw = len(df_train)
df_train = df_train.drop(df_train[(df_train.meter_reading > outl_up_meter_read_b) & (df_train.building_id == 1099)].index)
len_clean = len(df_train)
count_out = len_raw - len_clean
print('Jumlah outlier pada column meter_reading:',count_out)

In [ ]:
# Melakukan cek terhadap grafik rata-rata meter reading dari seteiap site
df_train_gb_site = df_train.groupby('site_id').meter_reading.mean().reset_index()
sns.barplot(x='site_id',y='meter_reading',data = df_train_gb_site)

Untuk data yang berkaitan dengan weather, tidak akan dibuang nilai outliernya, karena focus targetnya adalah variabel meter reading. 
Beberapa data yang berkaitan dengan weather juga tidak dapat dilakukan remove outliernya, karena:
1) Beberapa data mengindikasikan 0 sebagai nilai ketiadaan fenomena, sehingga apabila dilakukan remove outlier akan banyak data yang hilang

2) Beberapa data menghasilkan suatu nilai yang bukan merupakan suatu distribusi

In [ ]:
# Mencari tahu berapa banyak data yang hilang akibat dari dibuangnya outlier

data_clean_count = len_raw - len(df_train)
outlier_percentange = (1-(len(df_train))/len_raw)*100

print('Jumlah data yang hilang:',data_clean_count)
print('Persentase outlier yang telah dibuang:', outlier_percentange)

In [ ]:
print('Data train yang telah dilakukan preprocessing:')

df_train

In [ ]:
df_train.to_csv('eda_kelompok_a_df_train.csv', index = False)